In [267]:
%run data_loader.py

Data loaded


# Building Recommender Systems for Movie Rating Prediction

In this assignment, we will build a recommender systems that predict movie ratings. [MovieLense](https://grouplens.org/datasets/movielens/) has currently 25 million user-movie ratings.  Since the entire data is too big, we use  a 1 million ratings subset [MovieLens 1M](https://www.kaggle.com/odedgolden/movielens-1m-dataset), and we reformatted the data to make it more convenient to use.

In [268]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from sklearn.model_selection import train_test_split
from scipy.sparse import coo_matrix, csr_matrix
from scipy.spatial.distance import jaccard, cosine 
from pytest import approx
from tqdm import tqdm

In [269]:
MV_users = pd.read_csv('data/users.csv')
MV_movies = pd.read_csv('data/movies.csv')
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [270]:
from collections import namedtuple
Data = namedtuple('Data', ['users','movies','train','test'])
data = Data(MV_users, MV_movies, train, test)

### Starter codes
Now, we will be building a recommender system which has various techniques to predict ratings. 
The `class RecSys` has baseline prediction methods (such as predicting everything to 3 or to average rating of each user) and other utility functions. `class ContentBased` and `class Collaborative` inherit `class RecSys` and further add methods calculating item-item similarity matrix. You will be completing those functions using what we learned about content-based filtering and collaborative filtering.

`RecSys`'s `rating_matrix` method converts the (user id, movie id, rating) triplet from the train data (train data's ratings are known) into a utility matrix for 6040 users and 3883 movies.    
Here, we create the utility matrix as a dense matrix (numpy.array) format for convenience. But in a real world data where hundreds of millions of users and items may exist, we won't be able to create the utility matrix in a dense matrix format (For those who are curious why, try measuring the dense matrix self.Mr using .nbytes()). In that case, we may use sparse matrix operations as much as possible and distributed file systems and distributed computing will be needed. Fortunately, our data is small enough to fit in a laptop/pc memory. Also, we will use numpy and scipy.sparse, which allow significantly faster calculations than calculating on pandas.DataFrame object.    
In the `rating_matrix` method, pay attention to the index mapping as user IDs and movie IDs are not the same as array index.

In [271]:
class RecSys():
    def __init__(self,data):
        self.data=data
        self.allusers = list(self.data.users['uID'])
        self.allmovies = list(self.data.movies['mID'])
        self.genres = list(self.data.movies.columns.drop(['mID', 'title', 'year']))
        self.mid2idx = dict(zip(self.data.movies.mID,list(range(len(self.data.movies)))))
        self.uid2idx = dict(zip(self.data.users.uID,list(range(len(self.data.users)))))
        self.Mr=self.rating_matrix()
        self.Mm=None
        self.sim=np.zeros((len(self.allmovies),len(self.allmovies)))
        
    def rating_matrix(self):
        """
        Convert the rating matrix to numpy array of shape (#allusers,#allmovies)
        """
        ind_movie = [self.mid2idx[x] for x in self.data.train.mID] 
        ind_user = [self.uid2idx[x] for x in self.data.train.uID]
        rating_train = list(self.data.train.rating)
        
        return np.array(coo_matrix((rating_train, (ind_user, ind_movie)), shape=(len(self.allusers), len(self.allmovies))).toarray())


    def predict_everything_to_3(self):
        """
        Predict everything to 3 for the test data
        """
        # Generate an array with 3s against all entries in test dataset
        # your code here
        return np.ones(len(self.data.test.rating))*3
        
        
    def predict_to_user_average(self):
        """
        Predict to average rating for the user.
        Returns numpy array of shape (#users,)
        """

        # Generate an array as follows:
        # 1. Calculate all avg user rating as sum of ratings of user across all movies/number of movies whose rating > 0
        # 2. Return the average rating of users in test data
        # your code here

        mean_users = self.Mr.sum(axis=1)/(self.Mr>0).sum(axis=1)
        mean = [mean_users[self.uid2idx[uid]] for uid in self.data.test["uID"]]
        return np.array(mean)
    
    def predict_from_sim(self,uid,mid):
        """
        Predict a user rating on a movie given userID and movieID

        The algorithm is:
                   1. Get index of the provided user id (index_userID)
                   2. Get all the user ratings for the user using index_userID (ratings_index_userID)
                   3. Get index of the provided movie id (index_movieID)
                   4. Get all the similarity scores using index_movieID (movie_sims)
                   5. Take the **averaged** dot product.
        """
        # Predict user rating as follows:
        # 1. Get entry of user id in rating matrix
        # 2. Get entry of movie id in sim matrix
        # 3. Employ 1 and 2 to predict user rating of the movie

        
        # your code here
        uID = self.uid2idx[uid]
        mID = self.mid2idx[mid]
        
        user_ratings = self.Mr[uID]

        movie_simmilarity = self.sim[mID]
        
        return np.dot(movie_simmilarity,user_ratings)/np.dot(movie_simmilarity,user_ratings>0)
    
    def predict(self):
        """
        Predict ratings in the test data. Returns predicted rating in a numpy array of size (# of rows in testdata,)
        """
        # your code here
        yp = [self.predict_from_sim(uId, mId) for uId, mId in zip(self.data.test["uID"],self.data.test["mID"])]
        return np.array(yp)
    
    def rmse(self,yp):
        yp[np.isnan(yp)]=3 #In case there is nan values in prediction, it will impute to 3.
        yt=np.array(self.data.test.rating)
        return np.sqrt(((yt-yp)**2).mean())

    
class ContentBased(RecSys):
    def __init__(self,data):
        super().__init__(data)
        self.data=data
        self.Mm = self.calc_movie_feature_matrix()  
        
    def calc_movie_feature_matrix(self):
        """
        Create movie feature matrix in a numpy array of shape (#allmovies, #genres) 
        """
        # your code here
        movies_feature:pd.DataFrame = self.data.movies.set_index("mID", drop=True).drop(columns=["title","year"])
        
        return  movies_feature.to_numpy()
    
    def calc_item_item_similarity(self):
        """
        Create item-item similarity using Jaccard similarity
        """
        # Update the sim matrix by calculating item-item similarity using Jaccard similarity
        # Jaccard Similarity: J(A, B) = |A∩B| / |A∪B| 
        # your code here
        from tqdm import tqdm
        
        movies = self.mid2idx.values()
        for i in tqdm(movies):
            for j in movies:
                self.sim[i,j] = 1- jaccard(self.Mm[i],self.Mm[j])
                
class Collaborative(RecSys):    
    def __init__(self,data):
        super().__init__(data)
        
    def calc_item_item_similarity(self, simfunction, *X):  
        """
        Create item-item similarity using similarity function. 
        X is an optional transformed matrix of Mr
        """    
        # General function that calculates item-item similarity based on the sim function and data inputed
        if len(X)==0:
            self.sim = simfunction()            
        else:
            self.sim = simfunction(X[0]) # *X passes in a tuple format of (X,), to X[0] will be the actual transformed matrix
            
    def cossim(self):    
        """
        Calculates item-item similarity for all pairs of items using cosine similarity (values from 0 to 1) on utility matrix
        Returns a cosine similarity matrix of size (#all movies, #all movies)
        """
        # Return a sim matrix by calculating item-item similarity for all pairs of items using Jaccard similarity
        # Cosine Similarity: C(A, B) = (A.B) / (||A||.||B||) 
        # your code here
        
        user_mean_rating = self.Mr.sum(axis=1)/(self.Mr>0).sum(axis=1)
        mu_array = np.repeat(np.expand_dims(user_mean_rating, axis=1),self.Mr.shape[1],axis=1)
        X = self.Mr+(self.Mr==0)*mu_array-mu_array #Normalized movies rating
        Y = X/np.sqrt((X**2).sum(axis=0)) #Unitary normalized movies rating
        Y[np.isnan(Y)]=0. 
        cos = np.dot(Y.T,Y)
        for i in range(len(self.allmovies)):
            cos[i,i]=1
        return 0.5+0.5*cos
    def jacsim(self,Xr):
        """
        Calculates item-item similarity for all pairs of items using jaccard similarity (values from 0 to 1)
        Xr is the transformed rating matrix.
        """    
        # Return a sim matrix by calculating item-item similarity for all pairs of items using Jaccard similarity
        # Jaccard Similarity: J(A, B) = |A∩B| / |A∪B| 
        # your code here
        
        n = Xr.shape[1]
        maxr = int(Xr.max())
        if maxr>1:
            intersection = np.zeros((n,n)).astype(int)
            for i in range(1, maxr+1):
                csr = csr_matrix((Xr==i).astype(int))
                intersection += np.array(csr.T.dot(csr).toarray()).astype(int)    
        
        # Convert Xr into a CSR format
        csr0 = csr_matrix((Xr>0).astype(int))
        
        # Take the dot product
        nz_inter = np.array(csr0.T.dot(csr0).toarray()).astype(int)   
        
        # Formula JS:
        A = (Xr>0).astype(bool)
        rowsum = A.sum(axis=0)
        rsumtile = np.repeat(rowsum.reshape((n,1)),n,axis=1)   
        union = rsumtile.T + rsumtile - nz_inter

        if maxr>1:
            jac = intersection/union
        else:
            jac = nz_inter/union
            
        if np.isnan(jac).sum()>0:
            jac[np.isnan(jac)]=0
            for i in range(n):
                jac[i,i]=1   
        return jac        
    
    

# Q1. Baseline models [15 pts]

### 1a. Complete the function `predict_everything_to_3` in the class `RecSys`  [5 pts]

In [272]:
# Creating Sample test data
np.random.seed(42)
sample_train = train[:30000]
sample_test = test[:30000]


sample_MV_users = MV_users[(MV_users.uID.isin(sample_train.uID)) | (MV_users.uID.isin(sample_test.uID))]
sample_MV_movies = MV_movies[(MV_movies.mID.isin(sample_train.mID)) | (MV_movies.mID.isin(sample_test.mID))]


sample_data = Data(sample_MV_users, sample_MV_movies, sample_train, sample_test)

In [273]:
# Sample tests predict_everything_to_3 in class RecSys

sample_rs = RecSys(sample_data)
sample_yp = sample_rs.predict_everything_to_3()
print(sample_rs.rmse(sample_yp))
assert sample_rs.rmse(sample_yp)==approx(1.2642784503423288, abs=1e-3), "Did you predict everything to 3 for the test data?"

1.2642784503423288


### 1b. Complete the function predict_to_user_average in the class RecSys [10 pts]
Hint: Include rated items only when averaging

In [274]:
# Sample tests predict_to_user_average in the class RecSys
sample_yp = sample_rs.predict_to_user_average()
assert sample_rs.rmse(sample_yp)==approx(1.1429596846619763, abs=1e-3), "Check predict_to_user_average in the RecSys class. Did you predict to average rating for the user?" 

C:\Users\OM-FI\AppData\Local\Temp\ipykernel_6992\2214506112.py:44: RuntimeWarning: invalid value encountered in divide
  mean_users = self.Mr.sum(axis=1)/(self.Mr>0).sum(axis=1)


# Q2. Content-Based model

### 2a. Complete the function calc_movie_feature_matrix in the class ContentBased 

In [275]:
cb = ContentBased(data)
cb.Mm.shape

(3883, 18)

In [276]:
# tests calc_movie_feature_matrix in the class ContentBased 
assert(cb.Mm.shape==(3883, 18))

### 2b. Complete the function calc_item_item_similarity in the class ContentBased [10 pts]
This function updates `self.sim` and does not return a value.    
Some factors to think about:     
1. The movie feature matrix has binary elements. Which similarity metric should be used?
2. What is the computation complexity (time complexity) on similarity calcuation?      
Hint: You may use functions in the `scipy.spatial.distance` module on the dense matrix, but it is quite slow (think about the time complexity). If you want to speed up, you may try using functions in the `scipy.sparse` module. 

In [277]:
# cb.calc_item_item_similarity()

In [278]:
# Sample tests calc_item_item_similarity in ContentBased class 
sample_cb = ContentBased(sample_data)
print(sample_cb.Mm.shape)
sample_cb.calc_item_item_similarity() 

(3152, 18)


100%|██████████| 3152/3152 [01:54<00:00, 27.60it/s]


In [279]:
assert(sample_cb.sim.sum() > 0), "Check calc_item_item_similarity."
assert(np.trace(sample_cb.sim) == 3152), "Check calc_item_item_similarity. What do you think np.trace(cb.sim) should be?"


ans = np.array([[1, 0.25, 0.],[0.25, 1, 0.],[0., 0., 1]])
for pred, true in zip(sample_cb.sim[10:13, 10:13], ans):
    assert approx(pred, 0.01) == true, "Check calc_item_item_similarity. Look at cb.sim"

### 2c. Complete the function predict_from_sim in the class RecSys

In [280]:
# for a, b in zip(sample_MV_users.uID, sample_MV_movies.mID):
#     print(a, b, sample_cb.predict_from_sim(a,b))

# Sample tests for predict_from_sim in RecSys class 
assert(sample_cb.predict_from_sim(245,276)==approx(2.5128205128205128,abs=1e-2)), "Check predict_from_sim. Look at how you predicted a user rating on a movie given UserID and movieID."
assert(sample_cb.predict_from_sim(2026,2436)==approx(2.785714285714286,abs=1e-2)), "Check predict_from_sim. Look at how you predicted a user rating on a movie given UserID and movieID."

### 2d. Complete the function predict in the class RecSys
After completing the predict method in the RecSys class, run the cell below to calculate rating prediction and RMSE. How much does the performance increase compared to the baseline results from above? 

In [281]:
sample_yp = sample_cb.predict()
sample_rmse = sample_cb.rmse(sample_yp)
print(sample_rmse)

assert(sample_rmse==approx(1.1962537249116723, abs=1e-2)), "Check method predict in the RecSys class."

C:\Users\OM-FI\AppData\Local\Temp\ipykernel_6992\2214506112.py:73: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(movie_simmilarity,user_ratings)/np.dot(movie_simmilarity,user_ratings>0)


1.1962537249116723


In [282]:
# Hidden tests method predict in the RecSys class 

yp = cb.predict()
rmse = cb.rmse(yp)
print(rmse)

C:\Users\OM-FI\AppData\Local\Temp\ipykernel_6992\2214506112.py:73: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(movie_simmilarity,user_ratings)/np.dot(movie_simmilarity,user_ratings>0)


1.2585510334053043


# Q3. Collaborative Filtering

### 3a. Complete the function cossim in the class Collaborative
**To Do:**    
1.Impute the unrated entries in self.Mr to the user's average rating then subtract by the user mean, call this matrix X.   
2.Calculate cosine similarity for all item-item pairs. Don't forget to rescale the cosine similarity to be 0~1.    
You might encounter divide by zero warning (numpy will fill nan value for that entry). In that case, you can fill those with appropriate values.    

Hint: Let's say a movie item has not been rated by anyone. When you calculate similarity of this vector to anoter, you will get $\vec{0}$=[0,0,0,....,0]. When you normalize this vector, you'll get divide by zero warning and it will make nan value in self.sim matrix. Theoretically what should the similarity value for $\vec{x}_i \cdot \vec{x}_i$ when $\vec{x}_i = \vec{0}$? What about $\vec{x}_i \cdot \vec{x}_j$ when $\vec{x}_i = \vec{0}$ and $\vec{x}_j$ is an any vector?     

Hint: You may use `scipy.spatial.distance.cosine`, but it will be slow because its cosine function does vector-vector operation whereas you can implement matrix-matrix operation using numpy to calculate all cosines all at once (it can be 100 times faster than vector-vector operation in our data). Also pay attention to the definition. The scipy.spatial.distance provides distance, not similarity. 

3. Run the below cell that calculate yp and RMSE. 

In [283]:
# Sample tests cossim method in the Collaborative class

sample_cf = Collaborative(sample_data)
sample_cf.calc_item_item_similarity(sample_cf.cossim)
sample_yp = sample_cf.predict()
sample_rmse = sample_cf.rmse(sample_yp)

assert(np.trace(sample_cf.sim)==3152), "Check cossim method in the Collaborative class. What should np.trace(cf.sim) equal?"
assert(sample_rmse==approx(1.1429596846619763, abs=5e-3)), "Check cossim method in the Collaborative class. rmse result is not as expected."
assert(sample_cf.sim[0,:3]==approx([1., 0.5, 0.5],abs=1e-2)), "Check cossim method in the Collaborative class. cf.sim isn't giving the expected results."

C:\Users\OM-FI\AppData\Local\Temp\ipykernel_6992\2214506112.py:142: RuntimeWarning: invalid value encountered in divide
  user_mean_rating = self.Mr.sum(axis=1)/(self.Mr>0).sum(axis=1)
C:\Users\OM-FI\AppData\Local\Temp\ipykernel_6992\2214506112.py:73: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(movie_simmilarity,user_ratings)/np.dot(movie_simmilarity,user_ratings>0)


In [284]:
cf = Collaborative(data)
cf.calc_item_item_similarity(cf.cossim)
yp = cf.predict()
rmse = cf.rmse(yp)
print(rmse)

C:\Users\OM-FI\AppData\Local\Temp\ipykernel_6992\2214506112.py:145: RuntimeWarning: invalid value encountered in divide
  Y = X/np.sqrt((X**2).sum(axis=0)) #Unitary normalized movies rating


1.0263081874204125


### 3b. Complete the function jacsim in the class Collaborative


Function `jacsim` calculates jaccard similarity between items using collaborative filtering method. When we have a rating matrix `self.Mr`, the entries of Mr matrix are 0 to 5 (0: unrated, 1-5: rating). We are interested to see which threshold method works better when we use jaccard dimilarity in the collaborative filtering.    
We may treat any rating 3 or above to be 1 and the negatively rated (below 3) and no-rating as 0. Or, we may treat movies with any ratings to be 1 and ones that has no rating as 0. In this question, we will complete a function jacsim that takes a transformed rating matrix X and calculate and returns a jaccard similarity matrix.     
Let's consider these input cases for the utility matrix $M_r$ with ratings 1-5 and 0s for no-rating.    
1. $M_r \geq 3$ 
2. $M_r \geq 0$ 
3. $M_r$, no transform.

Things to think about: 
- The cases 1 and 2 are straightforward to calculate Jaccard, but what does Jaccard mean for multicategory data?
- Time complexity: The matrix $M_r$ is much bigger than the item feature matrix $M_m$, therefore it will take very long time if we calculate on dense matrix.     
Hint: Use sparse matrix.
- Which method will give the best performance?

### 3b-i)  When $M_r\geq3$
After you've implemented the jacsim function, run the code below. If implemented correctly, you'll have RMSE below 0.99. 

In [285]:
cf = Collaborative(data)
Xr = cf.Mr>=3
t0=time.perf_counter()
cf.calc_item_item_similarity(cf.jacsim,Xr)
t1=time.perf_counter()
time_sim = t1-t0
print('similarity calculation time',time_sim)
yp = cf.predict()
rmse = cf.rmse(yp)
print(rmse)
assert(rmse<0.99)

C:\Users\OM-FI\AppData\Local\Temp\ipykernel_6992\2214506112.py:183: RuntimeWarning: invalid value encountered in divide
  jac = nz_inter/union
C:\Users\OM-FI\AppData\Local\Temp\ipykernel_6992\2214506112.py:73: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(movie_simmilarity,user_ratings)/np.dot(movie_simmilarity,user_ratings>0)


similarity calculation time 1.1684768999984954
0.9819058692126349


### 3b-ii)  When $M_r\geq1$
After you've implemented the jacsim function, run the code below. If implemented correctly, you'll have RMSE below 1.0. 

In [286]:
cf = Collaborative(data)
Xr = cf.Mr>=1
t0=time.perf_counter()
cf.calc_item_item_similarity(cf.jacsim,Xr)
t1=time.perf_counter()
time_sim = t1-t0
print('similarity calculation time',time_sim)
yp = cf.predict()
rmse = cf.rmse(yp)
print(rmse)
assert(rmse<1.0)

C:\Users\OM-FI\AppData\Local\Temp\ipykernel_6992\2214506112.py:183: RuntimeWarning: invalid value encountered in divide
  jac = nz_inter/union
C:\Users\OM-FI\AppData\Local\Temp\ipykernel_6992\2214506112.py:73: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(movie_simmilarity,user_ratings)/np.dot(movie_simmilarity,user_ratings>0)


similarity calculation time 1.3540184000012232
0.991363571262366


### 3b-iii)  When $M_r$; no transform
After you've implemented the jacsim function, run the code below. If implemented correctly, you'll have RMSE below 0.96

In [287]:
cf = Collaborative(data)
Xr = cf.Mr.astype(int)
t0=time.perf_counter()
cf.calc_item_item_similarity(cf.jacsim,Xr)
t1=time.perf_counter()
time_sim = t1-t0
print('similarity calculation time',time_sim)
yp = cf.predict()
rmse = cf.rmse(yp)
print(rmse)
assert(rmse<0.96)

C:\Users\OM-FI\AppData\Local\Temp\ipykernel_6992\2214506112.py:181: RuntimeWarning: invalid value encountered in divide
  jac = intersection/union
C:\Users\OM-FI\AppData\Local\Temp\ipykernel_6992\2214506112.py:73: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(movie_simmilarity,user_ratings)/np.dot(movie_simmilarity,user_ratings>0)


similarity calculation time 3.175201799997012
0.9509126236828654


### 3.C Discussion
1. Summarize the methods and performances: 

|Method|RMSE|
|:----|:--------:|
|Baseline, $Y_p$=3| 1.2586|
|Baseline, $Y_p=\mu_u$| 1.0353|
|Content based, item-item| 1.0128|
|Collaborative, cosine| 1.0263|
|Collaborative, jaccard, $M_r\geq 3$| .9819|
|Collaborative, jaccard, $M_r\geq 1$|  .9914|
|Collaborative, jaccard, $M_r$|  .9509|

2. Discuss which method(s) work better than others and why.

Reading Material
1. First, start with a quick revision into terminologies by checking out:  [Quick Revision](https://www.youtube.com/watch?v=4MoSrMkWovM&ab_channel=DataMites)

2. Then, once you are there, let's deep dive into this by following these two videos:

[Deep Dive 1](https://www.youtube.com/watch?v=Blzp9iuhZqo)

[Deep Dive 2](https://www.youtube.com/watch?v=dfMoygb7FZE)

Why we need eigenvectors ? Visualize them [here](https://www.intmath.com/matrices-determinants/eigenvalues-eigenvectors-concept-applet.php).

3. Now, you should be equipped on how to go about Sparse Matrices multiplication and CSE conversions. And, here is everything combined on a Netflix Recommendation System case-study on how to implement a Dot Product for prediction:  [Using Dot Product in a Recommendation Engine](https://youtu.be/ZspR5PZemcs?t=515)

In case you are struggling: Read this excellent [Medium Article](https://medium.com/@chhavi.saluja1401/recommendation-systems-made-simple-b5a79cac8862#:~:text=The%20key%20notion%20here%20is%20to%20determine%20users%2C%20who%20are%20like%20the%20target%20user%20A%2C%20and%20recommend%20ratings%20for%20the%20unobserved%20ratings%20of%20A%20by%20calculating%20weighted%20averages%20of%20the%20ratings%20of%20this%20peer%20group)


[Here is an interesting repo](https://github.com/EthanJTucker/DTSA_5510_HW/blob/2829ceaf90d5659b4c925a9a6e3bdd297a2db73e/Week_3_HW/Programming_Assignment/Module3/Module3_USL_v3.ipynb#L4)
